<a href="https://colab.research.google.com/github/VickkiMars/AI-ML/blob/main/UltraChat_sft_qlora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [ ]:
def format_prompt(example):
  chat = example["messages"]
  prompt = tokenizer.apply_chat_template(chat, tokenize=False)
  return {"text": prompt}

In [ ]:
dataset = (
    load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft")
    .map(format_prompt)
    .shuffle(seed=42)
    .select(range(1000))
)

In [ ]:
print(dataset["text"][229])

<|user|>
Write a well-researched, informative article that delves into the benefits and drawbacks of using chatbots as a customer acquisition tool. Provide examples of businesses that have successfully implemented chatbots in their customer acquisition strategies, and discuss the best practices for creating and implementing effective chatbots. Additionally, explore the future of chatbots in customer acquisition and how they may evolve to better serve the needs of businesses and consumers. Use a professional, authoritative tone and provide credible sources to back up your claims.</s>
<|assistant|>
The chatbot market is expected to reach $9.4 billion by 2024, which demonstrates the increasing popularity of chatbots in the business world. Chatbots are computer programs powered by artificial intelligence that simulate conversation with humans. Chatbots are increasingly used for customer acquisition by businesses, since they can help in lead generation, lead conversion and nurturing.

Benef

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import bitsandbytes
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

model.config.use_cache = False
model.config.pretraininig_tp = 1

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ]
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"

training_arguments = TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    optim = "paged_adamw_32bit",
    learning_rate = 2e-4,
    lr_scheduler_type = "cosine",
    num_train_epochs=1,
    logging_steps=10,
    fp16 = True,
    gradient_checkpointing=True
)

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_arguments,
)

<ipython-input-15-8ba5ece9d35c>:3: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/1000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: martin_s (martin_s-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.479900
20,1.400000
30,1.308900
40,1.348100
50,1.296500
60,1.355600
70,1.254700
80,1.354000
90,1.333200
100,1.386500


TrainOutput(global_step=125, training_loss=1.3486832084655762, metrics={'train_runtime': 1057.9826, 'train_samples_per_second': 0.945, 'train_steps_per_second': 0.118, 'total_flos': 6588333383122944.0, 'train_loss': 1.3486832084655762})

In [ ]:
trainer.model.save_pretrained("TinyLlama-1.1B-UltraChat-qlora")